<h3><b> Based on this exercise, discuss the the questions in Quizizz with your group

Steps in a user-based recommendation system:

1. Select a user with the movies the user has watched
2. Based on his rating to movies, find the top x neighbours
3. Get the watched movie record of the user for each neighbour.
4. Calculate a similarity score using some formula
5. Recommend the items with the highest score

In [22]:
import pandas as pd
from math import sqrt
import numpy as np


In [23]:
animes_df = pd.read_csv('animes.csv')
ratings_df = pd.read_csv('ratings.csv')
animes_df.info()
animes_df



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15221 entries, 0 to 15220
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   anime_id    15221 non-null  int64  
 1   title       15221 non-null  object 
 2   genres      15218 non-null  object 
 3   media       15221 non-null  object 
 4   episodes    15221 non-null  int64  
 5   rating      11876 non-null  float64
 6   members     15205 non-null  float64
 7   start_date  15221 non-null  object 
 8   season      15221 non-null  object 
 9   source      12294 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 1.2+ MB


,anime_id,title,genres,media,episodes,rating,members,start_date,season,source
0,1,Cowboy Bebop,"Action,Adventure,Comedy,Drama,Sci-Fi,Space",tv,26,8.77,1324054.0,1998/4/3,spring_1998,original
1,5,Cowboy Bebop: Tengoku no Tobira,"Action,Drama,Mystery,Sci-Fi,Space",movie,1,8.39,284827.0,2001/9/1,summer_2001,original
2,6,Trigun,"Action,Sci-Fi,Adventure,Comedy,Drama,Shounen",tv,26,8.23,578887.0,1998/4/1,spring_1998,manga
3,7,Witch Hunter Robin,"Action,Mystery,Police,Supernatural,Drama,Magic",tv,26,7.27,96487.0,2002/7/2,summer_2002,original
4,8,Bouken Ou Beet,"Adventure,Fantasy,Shounen,Supernatural",tv,52,6.97,13450.0,2004/9/30,fall_2004,manga
...,...,...,...,...,...,...,...,...,...,...
15216,42392,Grisaia: Phantom Trigger The Animation - Starg...,"Action,School",movie,1,6.95,16328.0,2020/11/27,fall_2020,visual_novel
15217,42395,Shakunetsu Kabaddi,"Action,Comedy,School,Sports",tv,12,7.10,49112.0,2021/4/3,spring_2021,web_manga
15218,42397,Benkai to Ushiwakamaru,"Comedy,Historical,Samurai",movie,1,NaN,65.0,1999/12/4,fall_1999,NaN
15219,42399,Honnouji no Hen,"Historical,Samurai",movie,1,NaN,68.0,2000/3/18,winter_2000,NaN


In [35]:
userInput = [{'title':'Cowboy Bebop', 'rating':8.77},
             {'title':'Trigun', 'rating':8.23},
             {'title':'Witch Hunter Robin', 'rating':7.27},
             {'title':'Full Metal Panic!', 'rating':7.64},
             {'title':'Akira', 'rating':8.17}]
inputAnimes = pd.DataFrame(userInput)
print(inputAnimes)

                title  rating
0        Cowboy Bebop    8.77
1              Trigun    8.23
2  Witch Hunter Robin    7.27
3   Full Metal Panic!    7.64
4               Akira    8.17


In [36]:
inputId = animes_df[animes_df['title'].isin(inputAnimes['title'].tolist())]
inputAnimes = pd.merge(inputId, inputAnimes)
inputAnimes = inputAnimes[['anime_id','title','rating']]
print(inputAnimes)

   anime_id               title  rating
0         1        Cowboy Bebop    8.77
1         6              Trigun    8.23
2         7  Witch Hunter Robin    7.27
3        47               Akira    8.17
4        71   Full Metal Panic!    7.64


#### Now with the `movieId` in our input, we can now get the subset of users that have watched and reviewed the movies in our input. Find the similar user taste.

In [38]:
userSubset = ratings_df[ratings_df['anime_id'].isin(inputAnimes['anime_id'].tolist())]
print(userSubset.groupby('anime_id').count())

          user_id  rating
anime_id                 
1           11708   11708
6            7471    7471
7            1673    1673
47           7862    7862
71           8345    8345


In [39]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['user_id'])

def take_5_elem(x):
    # print (len(x[1]))
    return len(x[1])
    

#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup, key=take_5_elem, reverse=True)

userSubsetGroup = userSubsetGroup[0:100]
print(userSubsetGroup[0:5])


[(42,       user_id  anime_id  rating
2778       42        47       0
2885       42         1       0
2974       42        71       0
3564       42         6       0
3597       42         7       0), (48,       user_id  anime_id  rating
5753       48        47       8
5838       48         1       9
5904       48        71       8
6294       48         6       9
6307       48         7       8), (73,       user_id  anime_id  rating
9155       73        47       6
9243       73         1       9
9309       73        71       7
9669       73         6       9
9684       73         7       8), (150,        user_id  anime_id  rating
16776      150        47      10
16788      150         1      10
16797      150        71       6
16860      150         6       8
16864      150         7       8), (164,        user_id  anime_id  rating
18423      164        47       6
18453      164         1       7
18473      164        71       9
18642      164         6       8
18652      164         7 

In [40]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:

    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='anime_id')
    inputAnimes = inputAnimes.sort_values(by='anime_id')

    #Get the N for the formula
    nRatings = len(group)

    #Get the review scores for the movies that they both have in common
    temp_df = inputAnimes[inputAnimes['anime_id'].isin(group['anime_id'].tolist())]

    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
   
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
   
    
    #Now let's calculate the pearson correlation between two users, so called, x and y manually (check the formula from week 7 slide)
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0
    


In [41]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['user_id'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
print(pearsonDF.head())


   similarityIndex  user_id
0         0.000000       42
1         0.764530       48
2         0.343728       73
3         0.663889      150
4        -0.801636      164


In [42]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
print(topUsers.head())

    similarityIndex  user_id
62         0.969982     3520
47         0.937198     3033
42         0.921032     2868
27         0.921032     1950
40         0.914610     2697


In [45]:
topUsersRating=topUsers.merge(ratings_df, left_on='user_id', right_on='user_id', how='inner')
print(topUsersRating.head(100))

    similarityIndex  user_id  anime_id  rating
0          0.969982     3520      1549       7
1          0.969982     3520      2884       8
2          0.969982     3520       313       7
3          0.969982     3520      1292       8
4          0.969982     3520      4970       4
..              ...      ...       ...     ...
95         0.969982     3520      2025       8
96         0.969982     3520      4182       7
97         0.969982     3520      6573       6
98         0.969982     3520       974       9
99         0.969982     3520      6880       6

[100 rows x 4 columns]


In [46]:
#Multiplies the similarity by the user’s ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
print(topUsersRating.head())

   similarityIndex  user_id  anime_id  rating  weightedRating
0         0.969982     3520      1549       7        6.789873
1         0.969982     3520      2884       8        7.759855
2         0.969982     3520       313       7        6.789873
3         0.969982     3520      1292       8        7.759855
4         0.969982     3520      4970       4        3.879927


In [47]:
#Applies a sum to the topUsers after grouping it up by movieId
tempTopUsersRating = topUsersRating.groupby('anime_id').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
print(tempTopUsersRating.head())

          sum_similarityIndex  sum_weightedRating
anime_id                                         
1                   36.865057          348.916322
5                   30.367117          263.457101
6                   36.865057          326.401366
7                   36.865057          252.037294
8                    1.002261            4.208074


In [48]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()

#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['anime_id'] = tempTopUsersRating.index
print(recommendation_df.head(10))

          weighted average recommendation score  anime_id
anime_id                                                 
1                                      9.464690         1
5                                      8.675736         5
6                                      8.853950         6
7                                      6.836753         7
8                                      4.198582         8
15                                     7.002883        15
16                                     8.125057        16
17                                     5.000000        17
18                                     7.024955        18
19                                     7.814263        19


In [49]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
print(recommendation_df)


          weighted average recommendation score  anime_id
anime_id                                                 
1203                                       10.0      1203
29893                                      10.0     29893
15417                                      10.0     15417
1738                                       10.0      1738
39198                                      10.0     39198
...                                         ...       ...
10209                                       0.0     10209
23309                                       0.0     23309
1451                                        0.0      1451
10338                                       0.0     10338
5869                                        0.0      5869

[3552 rows x 2 columns]


In [51]:
recommended_movie=animes_df.loc[animes_df['anime_id'].isin(recommendation_df['anime_id'])]

#we don't want to recommend the same movie
recommended_movie=recommended_movie.loc[~recommended_movie.anime_id.isin(userSubset['anime_id'])]

print(recommended_movie)

       anime_id                                              title  \
1             5                    Cowboy Bebop: Tengoku no Tobira   
4             8                                     Bouken Ou Beet   
5            15                                       Eyeshield 21   
6            16                               Hachimitsu to Clover   
7            17                         Hungry Heart: Wild Striker   
...         ...                                                ...   
14929     41389                                    Tonikaku Kawaii   
14958     41468                                     Burn the Witch   
15007     41611                                      Wan Sheng Jie   
15124     42143                                        Aya to Majo   
15152     42203  Re:Zero kara Hajimeru Isekai Seikatsu 2nd Seas...   

                                         genres    media  episodes  rating  \
1             Action,Drama,Mystery,Sci-Fi,Space    movie         1    8.39   
4  